# These are experiments for
- django using Parse Trees
- django using BPE

The experimental setup is: using the same amount of data and the same model configuration what are the performance differences between the parse-tree tokenizer and a regularr BPE tokenizer

In [2]:
from src.dataset_loaders import CoNaLa_RawDataLoader, Django_RawDataLoader, Parseable_Django_RawDataLoader
from src.DataProcessors import Parse_Tree_Translation_DataProcessor, CodeTrainedBPE_Translation_DataProcessor
from src.translation_transformer import Translation_Transformer
from src.Experiments import TranslationExperiment
from src.tree_sitter_AST_utils import Node_Processor
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [4]:
%%capture
filtered_django_raw_data_loader = Parseable_Django_RawDataLoader()
train_pairs = filtered_django_raw_data_loader.english_to_code_for_translation("train")

## Pairs are now filtered to those that can be seen by the model
## Training with Parse Trees

In [11]:
train_dataloader = parse_tree_processor.to_dataloader(32, num_workers=0)

In [37]:
translator = Translation_Transformer(parse_tree_processor,
                                     parse_tree_processor.vocab_size, 
                                     embed_dim=256, 
                                     att_heads=4, 
                                     layers=4, 
                                     dim_feedforward=1024, 
                                     max_seq_length=550,
                                     use_copy=False)
translator.model.to("cuda")
translator.model.device

device(type='cuda', index=0)

In [13]:
translator.load("translator_Django_Parse_tree.pytorch")

In [ ]:
translator.train(100, train_dataloader)

## Training with BPE

In [5]:
BPE_processor = CodeTrainedBPE_Translation_DataProcessor(train_pairs)

In [7]:
train_dataloader = parse_tree_processor.to_dataloader(32, num_workers=0)

In [6]:
translator = Translation_Transformer(BPE_processor,
                                     BPE_processor.vocab_size, 
                                     embed_dim=256, 
                                     att_heads=4, 
                                     layers=4, 
                                     dim_feedforward=1024, 
                                     max_seq_length=550,
                                     use_copy=False)
translator.model.to("cuda")
translator.model.device

device(type='cuda', index=0)

In [7]:
translator.load("./translator_code_BPE_django.pytorch")

In [ ]:
translator.train(100, train_dataloader)

# Evaluating

In [8]:
translator.raw_batch_predict(["x was a string `a`", "foo is a list"])

['x =', 'foo']

In [18]:
def translation_batch_predictor(src_sents):
    outputs = translator.raw_batch_predict(src_sents, max_len=250)
    return [code for code, tree in outputs]

In [ ]:
test_pairs = filtered_django_raw_data_loader.english_to_code_for_translation("test")

In [29]:
Django_experiment = TranslationExperiment(test_pairs[:100])

In [31]:
Django_experiment.evaluate(translation_batch_predictor, batched=32, save_dir="./", save_name="translation_eval_Parse_tree.txt")

Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
T


saving translation eval to file: ./translation_eval_Parse_tree.txt
| EVALUATION | BLEU: 0.310 |


{'BLEU': 0.3101852808968385}

In [ ]:
0.3101852808968385